In [ ]:
# Setup for Google Colab (optional)
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    # Install required packages
    !pip install -q py4DSTEM hyperspy scikit-image matplotlib numpy scipy
    
    # Clone the repository to access data
    !git clone -q https://github.com/NU-MSE-LECTURES/465-WINTER2026.git
    import os
    os.chdir('/content/465-WINTER2026')
    
    # Set up file handling
    from google.colab import files
    print("Colab setup complete!")
else:
    print("Running in local environment")

<a href="https://colab.research.google.com/github/NU-MSE-LECTURES/465-WINTER2026/blob/main/Week_02/assignments/assignment_02_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 02: 4D-STEM Foundations

Complete this assignment to demonstrate your understanding of 4D-STEM data analysis and calibration.


In [1]:
# Colab setup
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab. Installing requirements...")
    !pip install hyperspy ase py4DSTEM
    !git clone https://github.com/NU-MSE-LECTURES/465_Computational_Microscopy_2026.git
    print("Setup complete.")
except ImportError:
    IN_COLAB = False
    print("Not running in Google Colab.")

Not running in Google Colab.


## Task 1: Distinguish Navigation vs. Signal Axes

In your notebook, define the "Navigation Axes" (where the measurement is made, e.g., x, y scan positions).

Define the "Signal Axes" (what is measured at each point, e.g., an EELS spectrum or a 2D diffraction pattern).

Use signal.axes manager to print and verify the dimensionality of a 4D-STEM dataset (expected: 2 Navigation, 2 Signal).

In [3]:
# Load necessary libraries
import py4DSTEM
import hyperspy.api as hs
import numpy as np

# Check py4DSTEM version
print(f"py4DSTEM version: {py4DSTEM.__version__}")
print(f"HyperSpy version: {hs.__version__}")

py4DSTEM version: 0.14.16
HyperSpy version: 2.3.0


In [7]:
# Create dummy array with correct dimensions
dummy_dataset = np.zeros((128,128,256,256))
signal_template = hs.signals.Signal2D(dummy_dataset)

# Define navigation axes
signal_template.axes_manager.navigation_axes[0].name = 'x'
signal_template.axes_manager.navigation_axes[1].name = 'y'

# Define signal axes
signal_template.axes_manager.signal_axes[0].name = 'qy'
signal_template.axes_manager.signal_axes[1].name = 'qx'

# Verify axes and dimensionality
print("Navigation axes:", len(signal_template.axes_manager.navigation_axes))
print("Signal axes:", len(signal_template.axes_manager.signal_axes))

Navigation axes: 2
Signal axes: 2


## Task 2: Load and Calibrate 4D-STEM Data

Use py4DSTEM.io.read to load a 4D-STEM dataset (e.g., .dm4 or .h5).

**Note:** The dataset Si-SiGe.dm4 should be available in the raw_data folder.

Set the scan step size (real space calibration) using dataset.set_scan_step_size().

Perform Center of Mass (CoM) correction using dataset.get_diffraction_shifts() to center the unscattered beam.

In [20]:
# Load dataset
filepath = "Si-SiGe.dm4"
dataset = py4DSTEM.io.import_file(filepath)

print("Dataset successfully loaded!")

# Inspect dataset
print("Dataset shape:", dataset.data.shape)
print(dataset.calibration)
print(dataset.metadata)

Dataset successfully loaded!
Dataset shape: (480, 448, 77, 17)
Calibration( A Metadata instance called 'calibration', containing the following fields:

             Q_pixel_size:    1
             R_pixel_size:    1
             Q_pixel_units:   pixels
             R_pixel_units:   pixels
             QR_flip:         False
)
{}


In [22]:
# Set scan step size
dataset.calibration.scan_pixel_size = 1.0 # nm
dataset.calibration.scan_pixel_units = "nm"

print("Scan step size set!")

# Verify calibration
print("Scan pixel size:", dataset.calibration.scan_pixel_size)
print("Scan pixel units:", dataset.calibration.scan_pixel_units)

Scan step size set!
Scan pixel size: 1.0
Scan pixel units: nm


In [30]:
# Verifying workflow (ChatGPT assisted)
import py4DSTEM.process.calibration as calib
[f for f in dir(calib) if "com" in f.lower() or "shift" in f.lower()]

['compare_QR_rotation', 'get_CoM']

In [31]:
# Confirming workflow (ChatGPT assisted)
from py4DSTEM.process.utils.utils import get_CoM

# pick one diffraction pattern at scan position (0,0)
dp = dataset.data[0,0,:,:]

qx, qy = get_CoM(dp)
print(qx, qy)

38.030162221042545 8.003136870988424


In [34]:
import numpy as np
from scipy.ndimage import shift
from py4DSTEM.process.utils.utils import get_CoM

# --- Step 1: Compute CoM shifts ---
qx_shift = np.zeros(dataset.shape[:2])
qy_shift = np.zeros(dataset.shape[:2])

for i in range(dataset.shape[0]):
    for j in range(dataset.shape[1]):
        dp = dataset.data[i, j, :, :]
        qx_shift[i, j], qy_shift[i, j] = get_CoM(dp)

# --- Step 2: Apply the shifts to each diffraction pattern ---
corrected = np.zeros_like(dataset.data)

for i in range(dataset.shape[0]):
    for j in range(dataset.shape[1]):
        dp = dataset.data[i, j, :, :]

        # Note: shift expects (y, x), so we reverse order
        corrected[i, j] = shift(dp, shift=(qy_shift[i, j], qx_shift[i, j]))

# Replace the dataset data with corrected data
dataset.data = corrected

print("✓ CoM correction applied (manual)")

✓ CoM correction applied (manual)


In [39]:
# Verification of workflow
print("qx shift stats:", qx_shift.min(), qx_shift.max())
print("qy shift stats:", qy_shift.min(), qy_shift.max())

qx shift stats: 28.61934027471672 54.815198482696815
qy shift stats: 7.811319587309935 8.24637859975725


## RED FLAG RESULTS -- NEED TO DEBUG OR RESTART

## Task 3: Virtual Detector Reconstruction

Generate a Virtual Bright Field (BF) image by integrating the central transmitted disk.

Generate an Annular Dark Field (ADF) image by integrating the scattered electrons in an outer ring.

Compare the Z-contrast in the ADF image to the diffraction contrast in the BF image.

In [ ]:
# Your code here

## Task 4: Basic 4D-STEM Visualization

Launch the interactive 4D-STEM browser using dataset.show() (if using a local GUI) or py4D.show_image().

Export a publication-quality figure of a virtual ADF image with a scale bar and a perceptually uniform colormap (e.g., magma).

In [2]:
# Your code here




from matplotlib.patches import Rectangle
if scale_bar_pixels < adf_image.shape[1] - 5:
    # Position scale bar in bottom-left corner
    bar_x, bar_y = 2, adf_image.shape[0] - 4
    scale_bar = Rectangle((bar_x, bar_y), scale_bar_pixels, 1, fill=True, color='white', linewidth=1)
    ax.add_patch(scale_bar)
    ax.text(bar_x + scale_bar_pixels/2, bar_y - 1, f'{scale_bar_length} nm', ha='center', va='top', 
            color='white', fontsize=10, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Integrated Intensity (a.u.)', fontsize=12)

plt.tight_layout()
plt.savefig('virtual_adf_figure_sisige.png', dpi=300, bbox_inches='tight')
plt.show()

## Task 5: Finalize and Submit

Update your README.md with a brief explanation of how virtual detectors allow post-acquisition imaging.

Push the completed Week 02 notebook to your GitHub repository.

Submit the repository link on Canvas.

In [ ]:
# Your code here